### Importing required libraries.

The data is collected from yahoo finance on a daily basis which can be altered depending on the requirement . 

In [ ]:
#Installing Yahoo Finance
#!pip install yfinance
#!pip install pandas_datareader
#!pip install Ta-Lib
#!brew install ta_lib
#!pip install tensortrade[talib]

In [1]:
import pandas as pd
import seaborn as sns
import joblib
import time, datetime
import numpy as np
%matplotlib inline
import matplotlib.pyplot as plt
import alpaca_trade_api as tradeapi
import os
import yfinance as yf
import pandas_datareader.data as pdr


### Importing Data from Yahoo Finance.

In [2]:
#Collecting desired ticker from user
ticker = input("Please enter your desired ticker")
start_time = datetime.datetime(1980,1,1)
end_time = datetime.datetime.now().date()

Please enter your desired ticker BTC


datetime.datetime.strptime(
        my_obj['dttm_utc'],
        '%Y-%m-%s %h:%m:%s'
    ).isoformat()

In [3]:
def get_data(ticker):
    df = pdr.get_data_yahoo(ticker, start=start_time, end=end_time)         
    return df

In [4]:
def compute_ema(df):
    df['EMA50'] = df['Close'].ewm(span=50, adjust=False).mean()
    #Setting adjust to False to specify that recursive calculation mode is required.
    df['EMA100'] = df['Close'].ewm(span=100, adjust=False).mean()
    #df['EMA150'] = EMA(df['Close'].values, timeperiod=150)
   # df['EMA200'] = EMA(df['Close'].values, timeperiod=200)
    return df


def computeMACD(df, n_fast, n_slow, n_smooth):
    data = df['Close']
    fastEMA = data.ewm(span=n_fast, min_periods=n_slow).mean()
    slowEMA = data.ewm(span=n_slow, min_periods=n_slow).mean()
    MACD = pd.Series(fastEMA-slowEMA, name = 'MACD')
    df = df.join(MACD)
    return df

In [5]:

df = get_data(ticker)

In [ ]:
#Computing the MACD indicator
#df =computeMACD(df, 12, 26, 9)
#The values can be altered along with the model. 
#The values are normalised between -1 to 1 which is not suitable for the current model
#As we are looking for the intersection of price and MACD.

In [7]:
df = compute_ema(df)

In [8]:
df.describe()

,High,Low,Open,Close,Volume,Adj Close,EMA50,EMA100
count,542.000000,542.000000,542.000000,542.000000,542.000000,542.000000,542.000000,542.000000
mean,96.866180,96.824366,96.848386,96.842862,3791.931734,96.842862,97.281350,97.646843
std,3.330815,3.344320,3.338969,3.337844,28343.094497,3.337844,2.963407,2.617771
min,88.609001,88.609001,88.609001,88.609001,0.000000,88.609001,90.197409,91.196817
25%,93.710749,93.653751,93.676250,93.681328,4.000000,93.681328,94.317914,95.497104
50%,98.639999,98.607250,98.623852,98.629700,108.500000,98.629700,98.981643,99.147750
75%,99.553751,99.452501,99.471802,99.516174,857.000000,99.516174,99.440937,99.405864
max,100.889999,100.154999,100.889999,100.154999,356863.000000,100.154999,100.000988,100.000497


### Creating the feature variables.

In [12]:
def calculated_features(df):
    df['aboveEMA50'] = np.where(df['Close'] > df['EMA50'], 1, 0)
    df['aboveEMA100'] = np.where(df['Close'] > df['EMA100'], 1, 0)
    return df

In [21]:
# create a new column in the DF based on the conditions
def defined_conditions(df):
    # List of conditions
    condition = [(df['aboveEMA50'] == df['aboveEMA100']) & (df['aboveEMA50'] ==1),
                 (df['aboveEMA50'] == df['aboveEMA100']) & (df['aboveEMA50'] ==0),
                 (df['aboveEMA50'] != df['aboveEMA100'])]
    # List of values to return
    choice  = [1,0,2]

    df['Buy/Sell'] = np.select(condition, choice, "ERROR")
    
    return df
#Considering it as a buy if the price is over 50 ema and 100 ema.
#Considering it as a Sell if the price is under 50 ema and 100 ema. 
#Further analysis is required if the price is over 50 ema and under ema 100 or vice versa.

### Creating DF for multiple stocks and Indices.

In [19]:
# Requesting the stock Data from the user. 
tickers = ['AAPL','GOOG','AMD','AMZN']

In [29]:

df = df.reset_index()
#Calling the get_data function definition. 
df = get_data(ticker)


#Calculating EMA 50 and 100
df = compute_ema(df)
#Calculating if the signal is to buy or sell.
df = calculated_features(df)
# Rules for placind a trade.
df = defined_conditions(df)


In [30]:
df

,High,Low,Open,Close,Volume,Adj Close,EMA50,EMA100,aboveEMA50,aboveEMA100,Buy/Sell
Date,,,,,,,,,,,
1997-05-15,0.125000,0.096354,0.121875,0.097917,1.443120e+09,0.097917,0.097917,0.097917,0,0,0
1997-05-16,0.098958,0.085417,0.098438,0.086458,2.940000e+08,0.086458,0.097468,0.097690,0,0,0
1997-05-19,0.088542,0.081250,0.088021,0.085417,1.221360e+08,0.085417,0.096995,0.097447,0,0,0
1997-05-20,0.087500,0.081771,0.086458,0.081771,1.093440e+08,0.081771,0.096398,0.097137,0,0,0
1997-05-21,0.082292,0.068750,0.081771,0.071354,3.770640e+08,0.071354,0.095416,0.096626,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...
2022-11-17,96.970001,94.029999,95.370003,94.849998,8.261790e+07,94.849998,108.850985,116.506075,0,0,0
2022-11-18,95.989998,92.480003,95.949997,94.139999,7.233650e+07,94.139999,108.274084,116.063183,0,0,0
2022-11-21,95.019997,90.589996,93.970001,92.459999,8.433030e+07,92.459999,107.653923,115.595793,0,0,0


In [26]:
df.corr()

,High,Low,Open,Close,Volume,Adj Close,EMA50,EMA100,aboveEMA50,aboveEMA100
High,1.000000,0.999803,0.999706,0.999879,-0.192174,0.999757,0.996759,0.993391,0.039459,0.060497
Low,0.999803,1.000000,0.999689,0.999881,-0.192571,0.999735,0.996508,0.993129,0.041094,0.061632
Open,0.999706,0.999689,1.000000,0.999589,-0.190239,0.999442,0.996548,0.993164,0.038770,0.059887
Close,0.999879,0.999881,0.999589,1.000000,-0.192371,0.999865,0.996563,0.993189,0.041163,0.061673
Volume,-0.192174,-0.192571,-0.190239,-0.192371,1.000000,-0.197047,-0.192481,-0.193523,0.051491,0.083476
Adj Close,0.999757,0.999735,0.999442,0.999865,-0.197047,1.000000,0.996506,0.993196,0.039798,0.059683
EMA50,0.996759,0.996508,0.996548,0.996563,-0.192481,0.996506,1.000000,0.998922,0.004353,0.027585
EMA100,0.993391,0.993129,0.993164,0.993189,-0.193523,0.993196,0.998922,1.000000,-0.008531,0.009452
aboveEMA50,0.039459,0.041094,0.038770,0.041163,0.051491,0.039798,0.004353,-0.008531,1.000000,0.765789
aboveEMA100,0.060497,0.061632,0.059887,0.061673,0.083476,0.059683,0.027585,0.009452,0.765789,1.000000
